In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import numpy as np
import folium 
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim

In [2]:
#!conda install -c conda-forge geopy --yes

## getting the data

In [3]:
ndf = pd.read_csv('Book6.csv')

In [4]:
ndf

,District,latitude,longitude
0,Al Izdihar,24.7803,46.7189
1,Al-Masif,24.7656,46.6803
2,Al-Mugharrazat,24.7617,46.7214
3,Al-Nakheel,24.7489,46.6527
4,Al-Malga,24.8143,46.6107
5,Hittin,24.7620,46.6038
6,Al-Wadi,24.7905,46.6919
7,Al-Ghadir,24.7734,46.6552
8,Al-Nafil,24.7830,46.6738
9,Al-Hamra,24.7783,46.7614


## Riyadh map

In [5]:
latitude = 24.7136
longitude= 46.6753

In [6]:
map_ry = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(ndf['latitude'], ndf['longitude'], ndf['District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ry)  
    
map_ry

## Foursquare API to explore and segment
## My Foursquare Credentials

In [13]:
CLIENT_ID = 'FVY3UUQJI5B1VTC2BMSHRMUTBEFUEHRLJO0BHYFNYIRGM2JQ' # my Foursquare ID
CLIENT_SECRET = 'ZZALLHVXFW0QRPGGV32Q0MLKPLXFS1PDEKBGQZW12W31S3XS' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FVY3UUQJI5B1VTC2BMSHRMUTBEFUEHRLJO0BHYFNYIRGM2JQ
CLIENT_SECRET:ZZALLHVXFW0QRPGGV32Q0MLKPLXFS1PDEKBGQZW12W31S3XS


In [14]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category' ]
    
    return(nearby_venues)

### Writting the code to run the function on each neighborhood and create a new dataframe called riyadh_venues.


In [16]:
riyadh_venues = getNearbyVenues(names=ndf['District'],
                                   latitudes=ndf['latitude'],
                                   longitudes=ndf['longitude']
                                    )

Al Izdihar
Al-Masif
Al-Mugharrazat
Al-Nakheel
Al-Malga
Hittin
Al-Wadi
Al-Ghadir
Al-Nafil
Al-Hamra
Qortubah
Al-Ra'id
King Fahd District
al-nada
al taawon
alflah


In [17]:
print(riyadh_venues.shape)
riyadh_venues.head(15)

(1600, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Al Izdihar,24.7803,46.7189,فطائر تركية,24.779479,46.719074,Turkish Restaurant
1,Al Izdihar,24.7803,46.7189,البقاله الي تفتح 24 ساعه,24.780144,46.719193,Market
2,Al Izdihar,24.7803,46.7189,Private Road Café (طريق خاص),24.775788,46.721389,Coffee Shop
3,Al Izdihar,24.7803,46.7189,Once Upon a Time Kids' Bookstore (مكتبة كان يا...,24.778667,46.711778,Bookstore
4,Al Izdihar,24.7803,46.7189,Knoll Coffee Roasters (محمصة و مقهى نول),24.776875,46.726181,Coffee Shop
5,Al Izdihar,24.7803,46.7189,Baskin Robbins (باسكن روبنز),24.778563,46.711553,Ice Cream Shop
6,Al Izdihar,24.7803,46.7189,Ame House Coffee,24.782824,46.720501,Coffee Shop
7,Al Izdihar,24.7803,46.7189,قرمز,24.776890,46.726151,Jewelry Store
8,Al Izdihar,24.7803,46.7189,Yplus,24.778762,46.710556,Coffee Shop
9,Al Izdihar,24.7803,46.7189,Baskin Robbins (باسكن روبنز),24.773404,46.723202,Ice Cream Shop


### Number of venues per District

In [18]:
riyadh_venues.groupby('District').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Al Izdihar,100,100,100,100,100,100
Al-Ghadir,100,100,100,100,100,100
Al-Hamra,100,100,100,100,100,100
Al-Malga,100,100,100,100,100,100
Al-Masif,100,100,100,100,100,100
Al-Mugharrazat,100,100,100,100,100,100
Al-Nafil,100,100,100,100,100,100
Al-Nakheel,100,100,100,100,100,100
Al-Ra'id,100,100,100,100,100,100


### Number of unique categories¶

In [19]:
print('There are {} uniques categories.'.format(len(riyadh_venues['Venue Category'].unique())))

There are 177 uniques categories.


### Analyzing Districts

In [20]:
# one hot encoding
ry_onehot = pd.get_dummies(riyadh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ry_onehot['District'] = riyadh_venues['District'] 

# move district column to the first column
cols=list(ry_onehot.columns.values)
cols.pop(cols.index('District'))
ry_onehot=ry_onehot[['District']+cols]

# rename Neighborhood for Districts so that future merge works
ry_onehot.rename(columns = {'District': 'District'}, inplace = True)
ry_onehot.head(15)

,District,Afghan Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,...,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Watch Shop,Wings Joint,Women's Store,Yoga Studio
0,Al Izdihar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,Al Izdihar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Al Izdihar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Al Izdihar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Al Izdihar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Al Izdihar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Al Izdihar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Al Izdihar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Al Izdihar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Al Izdihar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
ry_onehot.shape

(1600, 178)

### Grouping rows by district and by the mean of the frequency of occurrence of each category

In [22]:
ry_grouped = ry_onehot.groupby('District').mean().reset_index()
ry_grouped

,District,Afghan Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,...,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Watch Shop,Wings Joint,Women's Store,Yoga Studio
0,Al Izdihar,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00
1,Al-Ghadir,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00
2,Al-Hamra,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00
3,Al-Malga,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,...,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00
4,Al-Masif,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,Al-Mugharrazat,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00
6,Al-Nafil,0.00,0.01,0.00,0.01,0.00,0.00,0.01,0.00,0.00,...,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00
7,Al-Nakheel,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01
8,Al-Ra'id,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00
9,Al-Wadi,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,...,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00


In [17]:
ry_grouped.shape

(16, 178)

### Printing districts along with the top 5 most common venues

In [23]:
num_top_venues = 5

for hood in ry_grouped['District']:
    print("----"+hood+"----")
    temp = ry_grouped[ry_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Al Izdihar----
            venue  freq
0     Coffee Shop  0.21
1       Juice Bar  0.05
2  Cosmetics Shop  0.04
3    Dessert Shop  0.04
4        Pharmacy  0.03


----Al-Ghadir----
          venue  freq
0   Coffee Shop  0.12
1  Dessert Shop  0.07
2  Burger Joint  0.06
3          Café  0.05
4    Donut Shop  0.04


----Al-Hamra----
            venue  freq
0     Coffee Shop  0.11
1    Dessert Shop  0.09
2     Pizza Place  0.05
3        Pharmacy  0.05
4  Ice Cream Shop  0.04


----Al-Malga----
          venue  freq
0   Coffee Shop  0.11
1  Burger Joint  0.07
2   Supermarket  0.05
3          Café  0.05
4      Pharmacy  0.05


----Al-Masif----
                       venue  freq
0                Coffee Shop  0.19
1                 Donut Shop  0.06
2             Breakfast Spot  0.06
3  Middle Eastern Restaurant  0.05
4       Fast Food Restaurant  0.04


----Al-Mugharrazat----
          venue  freq
0   Coffee Shop  0.22
1     Juice Bar  0.05
2  Burger Joint  0.05
3    Donut Shop  0.04
4      

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
districts_venues_sorted = pd.DataFrame(columns=columns)
districts_venues_sorted['District'] = ry_grouped['District']

for ind in np.arange(ry_grouped.shape[0]):
    districts_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ry_grouped.iloc[ind, :], num_top_venues)

districts_venues_sorted

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Al Izdihar,Coffee Shop,Juice Bar,Dessert Shop,Cosmetics Shop,Pharmacy,Breakfast Spot,Shopping Mall,Chocolate Shop,Restaurant,Clothing Store
1,Al-Ghadir,Coffee Shop,Dessert Shop,Burger Joint,Café,Donut Shop,Ice Cream Shop,Gym / Fitness Center,Hotel,Furniture / Home Store,Middle Eastern Restaurant
2,Al-Hamra,Coffee Shop,Dessert Shop,Pizza Place,Pharmacy,Ice Cream Shop,Donut Shop,Gym / Fitness Center,Juice Bar,Furniture / Home Store,Bakery
3,Al-Malga,Coffee Shop,Burger Joint,Pharmacy,Café,Supermarket,Bakery,Restaurant,Food Truck,Flower Shop,Gift Shop
4,Al-Masif,Coffee Shop,Donut Shop,Breakfast Spot,Middle Eastern Restaurant,Fast Food Restaurant,Candy Store,Bookstore,Juice Bar,Café,Falafel Restaurant
5,Al-Mugharrazat,Coffee Shop,Burger Joint,Juice Bar,Bakery,Donut Shop,Supermarket,Dessert Shop,Pharmacy,Fast Food Restaurant,Pizza Place
6,Al-Nafil,Coffee Shop,Donut Shop,Dessert Shop,Café,Juice Bar,Burger Joint,Gym / Fitness Center,Bakery,Ice Cream Shop,Restaurant
7,Al-Nakheel,Coffee Shop,Bakery,Ice Cream Shop,Dessert Shop,Donut Shop,Grocery Store,Breakfast Spot,Café,Chocolate Shop,Jewelry Store
8,Al-Ra'id,Coffee Shop,Dessert Shop,Donut Shop,Middle Eastern Restaurant,Burger Joint,Food Truck,Hotel,Pizza Place,Plaza,Restaurant
9,Al-Wadi,Coffee Shop,Restaurant,Dessert Shop,Donut Shop,Café,Gym,Pharmacy,Ice Cream Shop,Supermarket,Plaza


## Clustering districts
## New dataframe for clusters and top 10 venues for each district
 ## Clustering Districts

In [26]:
# set number of clusters
kclusters = 5

ry_grouped_clustering = ry_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ry_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[1:10]

array([1, 1, 1, 0, 0, 1, 0, 1, 4], dtype=int32)

In [27]:
ry_merged = ndf

# add clustering labels
ry_merged['Cluster Labels'] = kmeans.labels_

# merge ry_grouped with ry_data to add latitude/longitude for each neighborhood
ry_merged = ry_merged.join(districts_venues_sorted.set_index('District'), on='District')
ry_merged # check the last columns!




,District,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Al Izdihar,24.7803,46.7189,0,Coffee Shop,Juice Bar,Dessert Shop,Cosmetics Shop,Pharmacy,Breakfast Spot,Shopping Mall,Chocolate Shop,Restaurant,Clothing Store
1,Al-Masif,24.7656,46.6803,1,Coffee Shop,Donut Shop,Breakfast Spot,Middle Eastern Restaurant,Fast Food Restaurant,Candy Store,Bookstore,Juice Bar,Café,Falafel Restaurant
2,Al-Mugharrazat,24.7617,46.7214,1,Coffee Shop,Burger Joint,Juice Bar,Bakery,Donut Shop,Supermarket,Dessert Shop,Pharmacy,Fast Food Restaurant,Pizza Place
3,Al-Nakheel,24.7489,46.6527,1,Coffee Shop,Bakery,Ice Cream Shop,Dessert Shop,Donut Shop,Grocery Store,Breakfast Spot,Café,Chocolate Shop,Jewelry Store
4,Al-Malga,24.8143,46.6107,0,Coffee Shop,Burger Joint,Pharmacy,Café,Supermarket,Bakery,Restaurant,Food Truck,Flower Shop,Gift Shop
5,Hittin,24.7620,46.6038,0,Food Truck,Coffee Shop,Café,Italian Restaurant,Plaza,French Restaurant,Restaurant,Tea Room,Bakery,Fast Food Restaurant
6,Al-Wadi,24.7905,46.6919,1,Coffee Shop,Restaurant,Dessert Shop,Donut Shop,Café,Gym,Pharmacy,Ice Cream Shop,Supermarket,Plaza
7,Al-Ghadir,24.7734,46.6552,0,Coffee Shop,Dessert Shop,Burger Joint,Café,Donut Shop,Ice Cream Shop,Gym / Fitness Center,Hotel,Furniture / Home Store,Middle Eastern Restaurant
8,Al-Nafil,24.7830,46.6738,1,Coffee Shop,Donut Shop,Dessert Shop,Café,Juice Bar,Burger Joint,Gym / Fitness Center,Bakery,Ice Cream Shop,Restaurant
9,Al-Hamra,24.7783,46.7614,4,Coffee Shop,Dessert Shop,Pizza Place,Pharmacy,Ice Cream Shop,Donut Shop,Gym / Fitness Center,Juice Bar,Furniture / Home Store,Bakery


## Clusters visualization of Riyadh's Districts

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ry_merged['latitude'], ry_merged['longitude'], ry_merged['District'], ry_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 0 - Most common venues: Coffee Shop and Food Truck	

In [39]:
ry_merged.loc[ry_merged['Cluster Labels'] == 0]

,District,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Al Izdihar,24.7803,46.7189,0,Coffee Shop,Juice Bar,Dessert Shop,Cosmetics Shop,Pharmacy,Breakfast Spot,Shopping Mall,Chocolate Shop,Restaurant,Clothing Store
4,Al-Malga,24.8143,46.6107,0,Coffee Shop,Burger Joint,Pharmacy,Café,Supermarket,Bakery,Restaurant,Food Truck,Flower Shop,Gift Shop
5,Hittin,24.7620,46.6038,0,Food Truck,Coffee Shop,Café,Italian Restaurant,Plaza,French Restaurant,Restaurant,Tea Room,Bakery,Fast Food Restaurant
7,Al-Ghadir,24.7734,46.6552,0,Coffee Shop,Dessert Shop,Burger Joint,Café,Donut Shop,Ice Cream Shop,Gym / Fitness Center,Hotel,Furniture / Home Store,Middle Eastern Restaurant
13,al-nada,24.8057,46.6844,0,Coffee Shop,Café,Burger Joint,Restaurant,Dessert Shop,Donut Shop,Plaza,Ice Cream Shop,Breakfast Spot,Sandwich Place


## Cluster 1 - Most common venues: Coffee Shop and Donut Shop		

In [40]:
ry_merged.loc[ry_merged['Cluster Labels'] == 1]

,District,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Al-Masif,24.7656,46.6803,1,Coffee Shop,Donut Shop,Breakfast Spot,Middle Eastern Restaurant,Fast Food Restaurant,Candy Store,Bookstore,Juice Bar,Café,Falafel Restaurant
2,Al-Mugharrazat,24.7617,46.7214,1,Coffee Shop,Burger Joint,Juice Bar,Bakery,Donut Shop,Supermarket,Dessert Shop,Pharmacy,Fast Food Restaurant,Pizza Place
3,Al-Nakheel,24.7489,46.6527,1,Coffee Shop,Bakery,Ice Cream Shop,Dessert Shop,Donut Shop,Grocery Store,Breakfast Spot,Café,Chocolate Shop,Jewelry Store
6,Al-Wadi,24.7905,46.6919,1,Coffee Shop,Restaurant,Dessert Shop,Donut Shop,Café,Gym,Pharmacy,Ice Cream Shop,Supermarket,Plaza
8,Al-Nafil,24.7830,46.6738,1,Coffee Shop,Donut Shop,Dessert Shop,Café,Juice Bar,Burger Joint,Gym / Fitness Center,Bakery,Ice Cream Shop,Restaurant


## Cluster 2 - Most common venues: Coffee Shop Dessert Shop

In [41]:
ry_merged.loc[ry_merged['Cluster Labels'] == 2]

,District,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Al-Ra'id,24.7092,46.6387,2,Coffee Shop,Dessert Shop,Donut Shop,Middle Eastern Restaurant,Burger Joint,Food Truck,Hotel,Pizza Place,Plaza,Restaurant


## Cluster 3 - Most common venues: Coffee Shop and Ice Cream Shop	

In [42]:
ry_merged.loc[ry_merged['Cluster Labels'] == 3]

,District,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Qortubah,24.8166,46.732,3,Coffee Shop,Ice Cream Shop,Plaza,Burger Joint,Pharmacy,Italian Restaurant,Donut Shop,Bakery,Middle Eastern Restaurant,Chinese Restaurant


## Cluster 4 - Most common venues: Coffee Shop and Pharmacys

In [43]:
ry_merged.loc[ry_merged['Cluster Labels'] == 4]

,District,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Al-Hamra,24.7783,46.7614,4,Coffee Shop,Dessert Shop,Pizza Place,Pharmacy,Ice Cream Shop,Donut Shop,Gym / Fitness Center,Juice Bar,Furniture / Home Store,Bakery
12,King Fahd District,24.7338,46.6744,4,Coffee Shop,Jewelry Store,Gift Shop,Pharmacy,Donut Shop,Furniture / Home Store,Breakfast Spot,Bakery,Salon / Barbershop,Lingerie Store
14,al taawon,24.7666,46.7051,4,Coffee Shop,Ice Cream Shop,Café,Pharmacy,Dessert Shop,Juice Bar,Hotel,Falafel Restaurant,Gift Shop,Donut Shop
15,alflah,24.7960,46.7101,4,Coffee Shop,Donut Shop,Gym / Fitness Center,Ice Cream Shop,Falafel Restaurant,Fast Food Restaurant,Restaurant,Hotel,Gift Shop,Dessert Shop
